In [13]:
import math
from scipy.stats import norm
import pandas as pd


In [18]:
# Set display precision for float values to 6 decimal places
pd.set_option('display.float_format', '{:.6f}'.format)

In [19]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
def LCG(N):
    uniformSamples = []
    x = 1
    a = 39373
    c = 0
    k = (2**31) - 1
    
    for i in range(N):
        x = ((a*x) + c)%k
        uniformSamples.append(x/k)
    
    return uniformSamples
        
    

In [3]:
"Check these functions again"

def phi(x):
    """CDF of standard normal distribution."""
    return 0.5 * (1 + math.erf(x / math.sqrt(2)))

def beaselySpringerMoro(u):
    # Constants
    a = [2.50662823884, -18.61500062529, 41.39119773534, -25.44106049637]
    b = [-8.47351093090, 23.08336743743, -21.06224101826, 3.13082909833]
    c = [0.3374754822726147, 0.9761690190917186, 0.1607979714918209,
         0.0276438810333863, 0.0038405729373609, 0.0003951896511919,
         0.0000321767881768, 0.0000002888167364, 0.0000003960315187]

    # Compute
    y = u - 0.5
    if abs(y) < 0.42:
        r = y * y
        x = y * (((a[3] * r + a[2]) * r + a[1]) * r + a[0]) / ((((b[3] * r + b[2]) * r + b[1]) * r + b[0]) * r + 1)
    else:
        r = u
        if y > 0:
            r = 1 - u
        r = math.log(-math.log(r))
        x = c[0] + r * (c[1] + r * (c[2] + r * (c[3] + r * (c[4] + r * (c[5] + r * (c[6] + r * (c[7] + r * c[8])))))))
        if y < 0:
            x = -x

    return x

In [27]:
def inverseTransform(N):
    uniformSamples = LCG(N)
    normalSamples = []
    for i,u in enumerate(uniformSamples):
        normalSamples.append(beaselySpringerMoro(u))
    
    return normalSamples

In [47]:
def blackScholesVanilla(K , T , S , v , q , r , CP ):
    phi = 1 if CP == 'C' else -1
    dp = (1/(v*math.sqrt(T)))*(math.log(S/K) + ((r - q + (v**2)/2)*T) )
    dm = dp - (v*math.sqrt(T))
    vBS = phi*((norm.cdf(phi*dp)*(S*math.exp(-q*T))) - ((norm.cdf(phi*dm))*(K*math.exp(-r*T))))
    return vBS

In [38]:
def downAndOutCall(K , B , T , S , v , q , r ):
    a = ((r - q)/(v**2)) - (1/2)
    CSK = blackScholesVanilla(K , T , S , v, q, r, 'C')
    CBSK = blackScholesVanilla(K , T , (B**2)/S , v, q, r , 'C')
    C_dao = CSK - (((B/S)**(2*a))*CBSK)
    return C_dao

In [7]:
normalSamples = inverseTransform(5120000)

In [35]:
K = 39
B = 36
S0 = 39
v = 0.25
r = 0.02
q = 0.01
T = 0.75

In [40]:
C_dao = downAndOutCall(K , B , T , S0 , v , q , r )
print(C_dao)

2.321408636702377


In [41]:
"Question 3.1"
df_result_3_1 = pd.DataFrame(columns = ['N_k' , 'm=200' , 'n' , 'V^(n)' , '|C_dao - V^(n)|'])
m = 200 # Discretization is fixed
k = list(range(10))
dT = T/m

for _k in k:
    n = 50*(2**_k)
    N_k = m*n
    nSamples = normalSamples[:N_k]
    ind = 0
    V_i = []
    for i in range(n):
        S = S0
        bHit = False
        for j in range(m):
            S = S*(math.exp(((r-q - (v**2)/2)*dT) + (v*math.sqrt(dT))*nSamples[ind]))
            ind += 1
            if S <= B:
                V_i.append(0)
                bHit = True
                break
        if not bHit:        
            V_i.append(max(S-K,0))
    
    #print(m ,dT ,  n ,len(V_i) ,N_k)
    #print(sum(V_i)/len(V_i))
    # Sample data
    data = {
        'N_k': N_k,  
        'm=200': 200,  # fixed value
        'n': n,  
        'V^(n)': sum(V_i)/len(V_i) ,  
        '|C_dao - V^(n)|': abs(sum(V_i)/len(V_i) -  C_dao)
    }

    # Append to dataframe
    df_result_3_1 = df_result_3_1.append(data, ignore_index=True)
        

In [42]:
"Question 3.2"
#m = 200 # Discretization is not fixed
df_result_3_2 = pd.DataFrame(columns = ['N_k' , 'm_k' , 'n_k' , 'V^(n_k)' , '|C_dao - V^(n_k)|'])
k = list(range(10))


for _k in k:
    #n = 50*(2**_k)
    N_k = 10000*(2**_k)
    nSamples = normalSamples[:N_k]
    m = math.ceil((N_k**(1/3))*(T**(2/3)))
    n = math.floor(N_k/m)
    dT = T/m
    ind = 0
    V_i = []
    for i in range(n):
        S = S0
        bHit = False
        for j in range(m):
            S = S*(math.exp(((r-q - (v**2)/2)*dT) + (v*math.sqrt(dT))*nSamples[ind]))
            ind += 1
            if S <= B:
                V_i.append(0)
                bHit = True
                break
        if not bHit:        
            V_i.append(max(S-K,0))
    #print(m , dT ,  n ,  len(V_i) , N_k)
    #print(sum(V_i)/len(V_i))# Sample data
    data = {
        'N_k': N_k,  
        'm_k': m,  
        'n_k': n,  
        'V^(n_k)': sum(V_i)/len(V_i),  
        '|C_dao - V^(n_k)|': abs(sum(V_i)/len(V_i) - C_dao)
    }

    # Append to dataframe
    df_result_3_2 = df_result_3_2.append(data, ignore_index=True)

In [43]:
df_result_3_1

,N_k,m=200,n,V^(n),|C_dao - V^(n)|
0,10000.000000,200.000000,50.000000,2.536046,0.214637
1,20000.000000,200.000000,100.000000,2.778498,0.457090
2,40000.000000,200.000000,200.000000,2.531745,0.210337
3,80000.000000,200.000000,400.000000,2.486791,0.165382
4,160000.000000,200.000000,800.000000,2.488276,0.166868
5,320000.000000,200.000000,1600.000000,2.299295,0.022114
6,640000.000000,200.000000,3200.000000,2.449582,0.128173
7,1280000.000000,200.000000,6400.000000,2.462477,0.141068
8,2560000.000000,200.000000,12800.000000,2.496556,0.175147
9,5120000.000000,200.000000,25600.000000,2.446702,0.125293


In [44]:
df_result_3_2

,N_k,m_k,n_k,V^(n_k),|C_dao - V^(n_k)|
0,10000.000000,18.000000,555.000000,2.861625,0.540216
1,20000.000000,23.000000,869.000000,2.920395,0.598986
2,40000.000000,29.000000,1379.000000,2.725195,0.403786
3,80000.000000,36.000000,2222.000000,2.588788,0.267379
4,160000.000000,45.000000,3555.000000,2.614974,0.293566
5,320000.000000,57.000000,5614.000000,2.475185,0.153776
6,640000.000000,72.000000,8888.000000,2.575492,0.254083
7,1280000.000000,90.000000,14222.000000,2.523672,0.202264
8,2560000.000000,113.000000,22654.000000,2.563242,0.241834
9,5120000.000000,143.000000,35804.000000,2.502472,0.181064


In [45]:
df_result = df_result_3_1.merge(df_result_3_2, on='N_k')

cols_to_convert = ['N_k', 'm=200', 'n', 'm_k', 'n_k']

for col in cols_to_convert:
    df_result[col] = df_result[col].astype(int)


In [46]:
df_result

,N_k,m=200,n,V^(n),|C_dao - V^(n)|,m_k,n_k,V^(n_k),|C_dao - V^(n_k)|
0,10000,200,50,2.536046,0.214637,18,555,2.861625,0.540216
1,20000,200,100,2.778498,0.457090,23,869,2.920395,0.598986
2,40000,200,200,2.531745,0.210337,29,1379,2.725195,0.403786
3,80000,200,400,2.486791,0.165382,36,2222,2.588788,0.267379
4,160000,200,800,2.488276,0.166868,45,3555,2.614974,0.293566
5,320000,200,1600,2.299295,0.022114,57,5614,2.475185,0.153776
6,640000,200,3200,2.449582,0.128173,72,8888,2.575492,0.254083
7,1280000,200,6400,2.462477,0.141068,90,14222,2.523672,0.202264
8,2560000,200,12800,2.496556,0.175147,113,22654,2.563242,0.241834
9,5120000,200,25600,2.446702,0.125293,143,35804,2.502472,0.181064
